In [99]:
import numpy as np
from scipy.special import comb

seed = 1
gen = np.random.default_rng(seed=seed)

n = 9
# 2^n random numbers
v = gen.uniform(0, 1, 2**n) # set function
p = gen.uniform(0, 1, n) # probability distribution
# store p as accurate as possible
print(p)

def compute_semivalue(v, p, i, n):
    semivalue_i = 0
    for set_idx in range(2**n):
        bin_string = bin(set_idx)[2:].zfill(n)
        size = bin_string.count('1')
        if bin_string[i] == '0':
            semivalue_i -= p[size] * v[set_idx]
        else:
            semivalue_i += p[size-1] * v[set_idx]
    return semivalue_i


[0.71702137 0.32012222 0.69361565 0.53859445 0.88773283 0.73336517
 0.40791874 0.48490952 0.47138576]


In [100]:
A = np.zeros((2**n, n))
for set_idx in range(2**n):
    bin_string = bin(set_idx)[2:].zfill(n)
    binary_vec = np.array([int(x) for x in bin_string])
    size = np.sum(binary_vec)
    ps = p[size] if size < n else 0
    ps1 = p[size-1] if size > 0 else 0

    A[set_idx] = binary_vec * ps1 - (1-binary_vec) * ps

# Compute on_diagonal of A^T A
on_diagonal = 0
for size in range(n+1):
    if 0 <= size and size <= n-1: # 0 to n-1
        on_diagonal += comb(n-1, size) * p[size]**2
    if 1 <= size and size <= n: # 1 to n
        on_diagonal += comb(n-1, size-1) * p[size-1]**2
# Compute off_diagonal of A^T A
off_diagonal = 0
for size in range(n+1):
    if 2 <= size and size <= n: # 2 to n
        off_diagonal += comb(n-2, size-2) * p[size-1]**2
    if 0 <= size and size <= n-2: # 0 to n-2
        off_diagonal += comb(n-2, size) * p[size]**2
    if 1 <= size and size <= n-1: # 1 to n-1
        off_diagonal += 2*comb(n-2, size-1) * -1 * p[size-1] * p[size]
scalar = on_diagonal - off_diagonal

# Solve argmin_x ||Ax - v||_2
x = np.linalg.lstsq(A, v, rcond=None)[0]
x = x * scalar
semivalues = [compute_semivalue(v, p, i, n) for i in range(n)]

In [101]:
i, j = 1, 2
semivalues[i] - semivalues[j], (x[i] - x[j])

(4.864632260488248, 4.86463226048828)